# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient()

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
client.list_database_names()

['admin', 'city_data', 'config', 'ironhack', 'local']

In [5]:
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [6]:
com = db.get_collection("companies")
com

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

In [7]:
type(com.find_one())

dict

In [8]:
com.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [9]:
for nam in com.find({'name': 'Babelgum'}):
    print(nam)

{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum', 'permalink': 'babelgum', 'crunchbase_url': 'http://www.crunchbase.com/company/babelgum', 'homepage_url': 'http://babelgum.com', 'blog_url': 'http://babelgum.com/blog', 'blog_feed_url': 'http://feeds.feedburner.com/Babelgum', 'twitter_username': 'Babelgum', 'category_code': 'games_video', 'number_of_employees': None, 'founded_year': 2007, 'founded_month': 3, 'founded_day': 1, 'deadpooled_year': 2013, 'deadpooled_month': 1, 'deadpooled_day': 9, 'deadpooled_url': '', 'tag_list': 'iptv, web2ireland', 'alias_list': '', 'email_address': 'info@babelgum.com', 'phone_number': '', 'description': '', 'created_at': 'Sat Jun 09 08:15:21 UTC 2007', 'updated_at': 'Wed Oct 16 06:30:25 UTC 2013', 'overview': '<p>Babelgum is an integrated web and mobile video content platform, free for users and supported by advertising, available on-demand to a global audience.</p>\n\n<p>On March 20th 2009 Babelgum released a U.S. version of its mobile v

In [10]:
com.count_documents({'name': 'Babelgum'})

1

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [11]:
proy = {"number_of_employees":1}
fl = list(com.find({"number_of_employees": {"$gte": 5000}},proy).limit(20))
fl

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'), 'number_of_employees': 5299},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9b'), 'number_of_employees': 15000},
 {'_id': ObjectId('52cdef7c4bab8bd675297da2'), 'number_of_employees': 63000},
 {'_id': ObjectId('52cdef7c4bab8bd675297da3'), 'number_of_employees': 13600},
 {'_id': ObjectId('52cdef7c4bab8bd675297dba'), 'number_of_employees': 28000},
 {'_id': ObjectId('52cdef7c4bab8bd675297dc4'), 'number_of_employees': 86300},
 {'_id': ObjectId('52cdef7c4bab8bd675297e49'), 'number_of_employees': 5080},
 {'_id': ObjectId('52cdef7c4bab8bd675297e5d'), 'number_of_employees': 7173},
 {'_id': ObjectId('52cdef7c4bab8bd675297e6f'), 'number_of_employees': 180500},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'), 'number_of_employees': 300000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e8e'), 'number_of_employees': 25000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e96'), 'number_of_employees': 8000},
 {'_id': ObjectId('52cdef7c4bab8bd675297ea4'), 'number_of_employee

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [12]:
project = {"name":1, "founded_year":1}
filt= {"founded_year": {"$gte":2000,"$lte":2005}}
results = db.companies.find(filt, project)

In [13]:
list(results[:5])

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_year': 2004},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'founded_year': 2004},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [14]:
db.com.find({"valuation_amount":{'$exists': 1}})

In [15]:
query = {"founded_year": {"$lt":2010}, "ipo.valuation_amount": {"$gt": 100000000}}
project = {"_id":0, "name":1, "ipo":1}
results = db.companies.find(query, project)
list(results[:1])

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [16]:
query = {"founded_year": {"$lt": 2005}, "number_of_employees":{"$lt": 1000}}
project = {"_id": 0, "name":1}
results = db.companies.find(query, project).sort("number_of_employees", 1).limit(10)
list(results[:10])

[{'name': 'Skype'},
 {'name': 'Simpy'},
 {'name': 'Eurekster'},
 {'name': 'Ticketmaster'},
 {'name': 'Fox Interactive Media'},
 {'name': 'stylediary'},
 {'name': 'Monster'},
 {'name': 'EditGrid'},
 {'name': 'Compete'},
 {'name': 'MindTouch'}]

### 6. All the companies that don't include the `partners` field.

In [17]:
resul = db.companies.find({"_id":{"$nin":["partners"]}},{"_id":0, "name":1})

In [18]:
list(resul[:10])

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Flektor'},
 {'name': 'Fox Interactive Media'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [19]:
query2 = list(db.companies.find({"category_code":None}, {"_id":0, "name":1}))                   
query2[:10]

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [20]:
query3 = {"number_of_employees":{"$gte":100, "$lt": 1000}}
project = {"_id":0, "name":1, "number_of_employees":1}
result = db.companies.find(query, project).sort("number_of_employees", 1)            
list(result[:10])

[{'name': 'Skype', 'number_of_employees': 0},
 {'name': 'Simpy', 'number_of_employees': 0},
 {'name': 'Eurekster', 'number_of_employees': 0},
 {'name': 'Ticketmaster', 'number_of_employees': 0},
 {'name': 'Fox Interactive Media', 'number_of_employees': 0},
 {'name': 'stylediary', 'number_of_employees': 0},
 {'name': 'Monster', 'number_of_employees': 0},
 {'name': 'EditGrid', 'number_of_employees': 0},
 {'name': 'Compete', 'number_of_employees': 0},
 {'name': 'MindTouch', 'number_of_employees': 0}]

### 9. Order all the companies by their IPO price in a descending order.

In [21]:
filt = {"ipo.valuation_amount":{"$exists":"ipo.valuation_amount"}}
project = {"_id":0,"name":1,"ipo.valuation_amount":1}
results = db.companies.find(filt, project).sort("ipo.valuation_amount", -1).limit(10)
list(results[:10])

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [22]:
project = {"_id":0, "name":1, "number_of_employees": 1}
filt = {"number_of_employees":{"$exists":"number_of_employees"}}
result = db.companies.find(filt, project).sort("number_of_employees", -1).limit(10)
list(result[:10])

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [23]:
filt = {"founded_month":{"$gte": 2, "$lte":6}}
project = {"_id": 0, "name": 1, "founded_month":1}
result = db.companies.find(filt, project).sort("founded_month", 1).limit(1000)
list(result[:5])

[{'name': 'StumbleUpon', 'founded_month': 2},
 {'name': 'Helio', 'founded_month': 2},
 {'name': 'YouTube', 'founded_month': 2},
 {'name': 'Prosper', 'founded_month': 2},
 {'name': 'Facebook', 'founded_month': 2}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [24]:
query = {"founded_year": {"$lt":2000}, "acquisition.price_amount": {"$gt": 1000000}}            
project = {"_id": 0, "name":1}
result = db.companies.find(query, project)
len(list(result[:]))

224

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [25]:
filt = {"acquisition.acquired_year": {"$lt":2010}}
project = {"_id":0, "name":1, "acquisition":1}
result = db.companies.find(filt, project).sort("acquisition.price_amount")
len(list(result[:]))

1554

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [26]:
filt = {"founded_year":{"$exists":["founded_year"]}}
project = {"_id":0, "name":1, "founded_year": 1}
result = db.companies.find(filt,project).sort("founded_year", -1)

list(result[:5])

[{'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [27]:
filt = {"founded_day": {"$lte": 7, "$gte":1}}
project = {"_id": 0, "name":1}
result = db.companies.find(filt, project).sort("acquisition.price_amount")
list(result[:10])
# No sé cómo poner el "reverse = True" aquí para sort en descending order

[{'name': 'eBay'},
 {'name': 'Joost'},
 {'name': 'Fox Interactive Media'},
 {'name': 'Geni'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Technorati'},
 {'name': 'Yahoo!'},
 {'name': 'Babelgum'},
 {'name': 'Gizmoz'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [28]:
filt = {"category_code": {"$eq": "web"}, "number_of_employees":{"$gt": 4000}}
project = {"_id": 0, "name": 1}
results = db.companies.find(filt, project).sort("number_of_employees")
list(results[:6])

[{'name': 'Expedia'},
 {'name': 'AOL'},
 {'name': 'Webkinz'},
 {'name': 'Rakuten'},
 {'name': 'Groupon'},
 {'name': 'Los Angeles Times Media Group'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [37]:
filt ={"acquisition.price_amount": {"$gt": 10000000}, "acquisition.price_currency_code": {"$eq": "EUR"}}
project = {"_id": 0, "name": 1}
results = db.companies.find(filt, project)
list(results[:6])

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [38]:
query = {"acquisition.acquired_month": {"$lte": 3}}
projects = {"_id":0, "name":1, "acquisition":1}
results = db.companies.find(query, projects)
list(results[:10])

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [45]:
filt = {"founded_year":{"$lte":2010, "$gte": 2000}, "acquisition.acquired_year": {"$gt": 2011}}
params = {"_id": 0, "name":1, "founded_year": 1, "acquisition.acquired_year":1}
results = db.companies.find(filt, params).sort("acquisition.acquired_year")
list(results[:6])

[{'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Revision3',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Netvibes',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'KickApps',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'iContact',
  'founded_year': 2003,
  'acquisition': {'acquired_year': 2012}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [46]:
filt = {"deadpooled_year": {"$gt":3}}
project = {"_id": 0, "name": 1, "deadpooled_year":1}
results = db.companies.find(filt, project).limit(10)
list(results[:5])

[{'name': 'Omnidrive', 'deadpooled_year': 2008},
 {'name': 'Babelgum', 'deadpooled_year': 2013},
 {'name': 'Sparter', 'deadpooled_year': 2008},
 {'name': 'Thoof', 'deadpooled_year': 2013},
 {'name': 'Mercora', 'deadpooled_year': 2008}]